<a href="https://colab.research.google.com/github/syou2020/colab/blob/master/tputest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import pprint
import tensorflow as tf

if 'COLAB_TPU_ADDR' not in os.environ:
  print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
else:
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print ('TPU address is', tpu_address)

  with tf.Session(tpu_address) as session:
    devices = session.list_devices()
    
  print('TPU devices:')
  pprint.pprint(devices)

In [2]:
# TFLearnのインストール
!pip install tflearn

In [3]:
# 使用するライブラリのインポート
import tflearn
import tflearn.datasets.mnist as mnist
import tensorflow as tf
import numpy as np
 
# MNISTのデータを./mnistへダウンロード＆各変数へ値を格納
X,Y,testX,testY = mnist.load_data('mnist',one_hot=True)

Extracting mnist/train-images-idx3-ubyte.gz
Extracting mnist/train-labels-idx1-ubyte.gz
Extracting mnist/t10k-images-idx3-ubyte.gz
Extracting mnist/t10k-labels-idx1-ubyte.gz


In [4]:
%%timeit
# おまじない
tf.reset_default_graph()

# 入力層の作成
net = tflearn.input_data(shape=[None, 784])

# 中間層を３層作成
net = tflearn.fully_connected(net, 20, activation = 'sigmoid', weight_decay = 0.001)
net = tflearn.fully_connected(net, 20, activation = 'sigmoid', weight_decay = 0.001)
net = tflearn.fully_connected(net, 20, activation = 'sigmoid', weight_decay = 0.001)

# 出力層の作成
net = tflearn.fully_connected(net, 10, activation = 'softmax')

# 学習条件の設定
net = tflearn.regression(net, optimizer='adam', loss = 'categorical_crossentropy')

model = tflearn.DNN(net, tensorboard_verbose = 0)

# トレーニング
model.fit(X,Y,n_epoch = 20, batch_size=128 , shuffle=True, validation_set = (testX, testY))

# テスト
predictions = np.array(model.predict(testX)).argmax(axis=1)
actual = testY.argmax(axis=1)
test_accuracy = np.mean(predictions == actual, axis=0)
print("Test accuracy: %f" %test_accuracy)

Training Step: 8599  | total loss: 0.44857 | time: 2.673s
| Adam | epoch: 020 | loss: 0.44857 -- iter: 54912/55000
Training Step: 8600  | total loss: 0.42104 | time: 3.715s
| Adam | epoch: 020 | loss: 0.42104 | val_loss: 0.26122 -- iter: 55000/55000
--
Test accuracy: 0.933600
1 loop, best of 3: 1min 37s per loop
